In [1]:
import argparse
import csv
import distutils.util
import distutils
import os
import Dummy
import numpy as np
import pandas as pd
from PIL import Image
import os



In [2]:
# convert Dataset from pandas dataframe to numpy array and dictionary 
def convert_to_ndarray(data):
    data_dict = {}
    for column in data.columns:
        data_dict[column] = data[column].to_numpy()
    data_ndarray = np.column_stack(list(data_dict.values()))
    return data_dict,data_ndarray

    

In [3]:
classification_scheme = ['Female', 'Male', 'Primate', 'Rodent', 'Food']


In [4]:
train_df = pd.read_csv("E:/Upwork project/training_data.csv")
test_df = pd.read_csv("E:/Upwork project/testing_data.csv")


In [5]:
train_dict , train_ndarray = convert_to_ndarray(train_df)
test_dict , test_ndarray = convert_to_ndarray(test_df)

In [6]:
def validateDataFormat(data, predicted):
    # Define the classification scheme
#     classification_scheme = ['Female', 'Male', 'Primate', 'Rodent', 'Food']
    for key, value in data.items():
        size = value.size
    # Check if data is empty
    if size == 0:
        return False

    # Check if the required columns are present
    if predicted:
        required_columns = ["Path", "ActualClass", "PredictedClass"]
    else:
        required_columns = ["Path", "ActualClass"]
    if not all(col in data.keys() for col in required_columns):
        return False 

    # Check if values in the "Path" column are file paths
    if not all(isinstance(path, str) for path in data["Path"]):
        return False

    # Check if values in the "ActualClass" column are from the classification scheme
    if not all(actual_class in classification_scheme for actual_class in data["ActualClass"]):
        return False

    # Check if values in the "PredictedClass" column (if present) are from the classification scheme
    if predicted:
        if not all(predicted_class in classification_scheme for predicted_class in data["PredictedClass"]):
            return False

    # Check if the number of entries in "Path" matches the number of entries in "ActualClass" (and "PredictedClass" if predicted is True)
    if len(data["Path"]) != len(data["ActualClass"]):
        return False
    if predicted:
        if len(data["Path"]) != len(data["PredictedClass"]):
            return False
        
    return True


In [7]:
def readAndResize(image_path, width=60, height=30):
   
    parts = image_path.split('/')
    part1_path = "E:/Upwork project/images/"

    # Extract the desired part from the second element onwards
    desired_part = '/'.join(parts[-2:])
    final = part1_path + desired_part
    try:
        # Open the image file
        with Image.open(final) as img:
            # Resize the image
            resized_img = img.resize((width, height))
            # Convert the image to numpy array
            image = np.array(resized_img)
    except FileNotFoundError:
        print("Image not found at the given path.")
        # Return an empty array if the image is not found
        image = np.array([])

    return image

In [8]:
#Euclidean Distance 
def computeMeasure1(image1, image2):
    # Write here what kind of measure you are using!
     # Check if images are empty
    if image1.size == 0 or image2.size == 0:
        value = float('nan')
        return value


    # Check if images have the same shape
    if image1.shape != image2.shape:
        raise ValueError("Input images must have the same shape")

    # Calculate Euclidean distance
    distance = np.linalg.norm(image1 - image2)

    return distance

# Manhattan distance    
def computeMeasure2(image1, image2):
    # Write here what kind of measure you are using!
     # Check if images are empty
    if image1.size == 0 or image2.size == 0:
        value = float('nan')
        return value

    # Check if images have the same shape
    if image1.shape != image2.shape:
        raise ValueError("Input images must have the same shape")
    # Calculate Manhattan distance
    distance = np.sum(np.abs(image1 - image2))
    return distance

# cosine similarity
def computeMeasure3(image1, image2):
    # Write here what kind of measure you are using!
    # Check if images are empty
    if image1.size == 0 or image2.size == 0:
        value = float('nan')
        return value

    # Check if images have the same shape
    if image1.shape != image2.shape:
        raise ValueError("Input images must have the same shape")
    # Compute dot product and norms without flattening
    dot_product = np.sum(image1 * image2)
    norm_image1 = np.linalg.norm(image1)
    norm_image2 = np.linalg.norm(image2)

    # Calculate cosine similarity
    similarity = dot_product / (norm_image1 * norm_image2)

    return similarity





In [9]:
# euclidean_distance from scratch
def selfComputeMeasure1(image1, image2):
    # Write here what kind of measure you are using - it has to correspond to the approach in computeMeasure1
        # Check if images are empty
    if image1.size == 0 or image2.size == 0:
        value = float('nan')
    return value

    # Check if images have the same shape
    if image1.shape != image2.shape:
        raise ValueError("Input images must have the same shape")

    # Initialize squared differences
    squared_diff_sum = 0

    # Iterate through pixels and calculate squared differences
    for i in range(image1.shape[0]):
        for j in range(image1.shape[1]):
            for k in range(image1.shape[2]):
                squared_diff_sum += (image1[i, j, k] - image2[i, j, k]) ** 2

    # Compute square root of the sum of squared differences
    euclidean_distance = np.sqrt(squared_diff_sum)

    return euclidean_distance

# manhattan_distance from scratch
def selfComputeMeasure2(image1, image2):
    # Write here what kind of measure you are using - it has to correspond to the approach in computeMeasure2
    # Check if images are empty
    if image1.size == 0 or image2.size == 0:
        value = float('nan')

    return value

    # Check if images have the same shape
    if image1.shape != image2.shape:
        raise ValueError("Input images must have the same shape")

    # Initialize Manhattan distance
    manhattan_distance = 0

    # Iterate through pixels and calculate Manhattan distance
    for i in range(image1.shape[0]):
        for j in range(image1.shape[1]):
            for k in range(image1.shape[2]):
                manhattan_distance += abs(image1[i, j, k] - image2[i, j, k])

    return manhattan_distance
   




In [10]:
def getClassesOfKNearestNeighbours(measure_classes, k, similarity_flag):
    nearest_neighbours_classes = {}

    # Sort the measure_classes based on distance or similarity
    if similarity_flag ==True:
        sorted_measure_classes = sorted(measure_classes, key=lambda x: x[0], reverse=True)
    else:
        sorted_measure_classes = sorted(measure_classes, key=lambda x: x[0])

    # Get the k nearest neighbours
    k_nearest_neighbours = sorted_measure_classes[:k]

    # Count the occurrences of each class
    for measure, class_label in k_nearest_neighbours:
        if class_label in nearest_neighbours_classes:
            nearest_neighbours_classes[class_label] += 1
        else:
            nearest_neighbours_classes[class_label] = 0

    return nearest_neighbours_classes


In [11]:
def getMostCommonClass(nearest_neighbours_classes):
    winner = 'NaN'
     # Check if the input dictionary is empty or if it does not contain any classes from the scheme
    if not nearest_neighbours_classes or set(nearest_neighbours_classes.keys()).isdisjoint(classification_scheme):
        return winner
    
    # Check if all classes in the scheme have occurrence of 0
    if all(count == 0 for count in nearest_neighbours_classes.values()):
        return winner
    winner = max(nearest_neighbours_classes, key=nearest_neighbours_classes.get)

    return winner


In [12]:
def kNN(training_data, k, measure_func, similarity_flag, data_to_classify,getMostCommonClass, getClassesOfKNearestNeighbours,readAndResize12):
    # This sets the header list
    classified_data = np.array([['Path', 'ActualClass', 'PredictedClass']])
    # Iterate over each data point in data_to_classify
    for data_point in test_ndarray:
        # Extract the path and actual class from the data point
        test_path, test_actual_class = data_point[:2]
        
        # Apply the read_func to read and resize the image (assuming it returns empty array for invalid paths)
        test_image = readAndResize12(test_path, width=60, height=30)
        
        # Check if the image is empty (i.e., path is invalid)
        if test_image.size == 0:
            continue  # Skip this data point
        
        # Calculate distances/similarities between the current data point and all data points in training_data
        measures_classes = []
        for training_point in train_ndarray:
            train_path, train_actual_class = training_point[:2]
            train_image = readAndResize12(train_path, width=60, height=30)
            if train_image.size == 0:
                continue  # Skip this train data point
             
            measure = measure_func(train_image, test_image)
            measures_classes.append((measure, train_actual_class)) 
   
            
        # Get the classes of k nearest neighbors
        nearest_neighbours_classes = getClassesOfKNearestNeighbours(measures_classes, k, similarity_flag)
        
        # Get the predicted class for the current data point
        predicted_class = getMostCommonClass(nearest_neighbours_classes)
        
        # Append the path, actual class, and predicted class to the classified_data array
        classified_data = np.append(classified_data, [[test_path, test_actual_class, predicted_class]], axis=0)
    
    # Have fun!
    
    return classified_data



In [13]:
classified_data = kNN(train_ndarray , 11 , computeMeasure2, False , test_ndarray , getMostCommonClass , getClassesOfKNearestNeighbours , readAndResize)

In [14]:
classified_data

array([['Path', 'ActualClass', 'PredictedClass'],
       ['..\\Images/Student_Test/generated.photos_v3_0626369.jpg',
        'Female', 'Female'],
       ['..\\Images/Student_Test/generated.photos_v3_0081097.jpg',
        'Female', 'Rodent'],
       ['..\\Images/Student_Test/generated.photos_v3_0405408.jpg',
        'Female', 'Rodent'],
       ['..\\Images/Student_Test/generated.photos_v3_0582035.jpg',
        'Female', 'Female'],
       ['..\\Images/Student_Test/generated.photos_v3_0090463.jpg',
        'Female', 'Female'],
       ['..\\Images/Student_Test/generated.photos_v3_0899088.jpg',
        'Female', 'Female'],
       ['..\\Images/Student_Test/generated.photos_v3_0299741.jpg',
        'Female', 'Female'],
       ['..\\Images/Student_Test/generated.photos_v3_0083465.jpg',
        'Female', 'Female'],
       ['..\\Images/Student_Test/generated.photos_v3_0194016.jpg',
        'Female', 'Female'],
       ['..\\Images/Student_Test/generated.photos_v3_0548166.jpg',
        'Female', '

# Basic evaluation

In [15]:
def confusion_matrix(classified_data):
    confusion_matrix = np.zeros((len(classification_scheme), len(classification_scheme)), dtype=int)
    
    for row in classified_data[1:] : # Skip the first row which contains column headers
        actual_class = row[1]
        predicted_class = row[2]
        actual_index = classification_scheme.index(actual_class)
        predicted_index = classification_scheme.index(predicted_class)
        confusion_matrix[predicted_index, actual_index] += 1
    sorted_indices = np.argsort(classification_scheme)
    sorted_confusion_matrix = confusion_matrix[sorted_indices][:, sorted_indices]

    return sorted_confusion_matrix


In [16]:
def computeTPs(confusion_matrix):
    tps = [confusion_matrix[classification_scheme.index(cls), classification_scheme.index(cls)] for cls in classification_scheme]
    return tps

def computeFPs(confusion_matrix):
    fps = []
    for cls in classification_scheme:
        col_idx = classification_scheme.index(cls)
        fp = sum(confusion_matrix[:, col_idx]) - confusion_matrix[col_idx, col_idx]
        fps.append(fp)
    return fps

def computeFNs(confusion_matrix):
    fns = []
    for cls in classification_scheme:
        row_idx = classification_scheme.index(cls)
        fn = sum(confusion_matrix[row_idx, :]) - confusion_matrix[row_idx, row_idx]
        fns.append(fn)
    return fns


In [17]:
def computeMacroPrecision(tps, fps, fns, data_size):
    precisions = []
    for i in range(len(tps)):
        if (tps[i] + fps[i]) != 0:
            precision = tps[i] / (tps[i] + fps[i])
        else:
            precision = 0
        precisions.append(precision)
    macro_precision = sum(precisions) / len(precisions)
    return macro_precision

def computeMacroRecall(tps, fps, fns, data_size):
    recalls = []
    for i in range(len(tps)):
        if (tps[i] + fns[i]) != 0:
            recall = tps[i] / (tps[i] + fns[i])
        else:
            recall = 0
        recalls.append(recall)
    macro_recall = sum(recalls) / len(recalls)
    return macro_recall

def computeMacroFMeasure(computeMacroRecall, computeMacroPrecision):
    macro_f_measure = (2*computeMacroPrecision*computeMacroRecall)/(computeMacroPrecision+computeMacroRecall)
    return macro_f_measure

def computeAccuracy(tps, fps, fns, data_size):
    accuracies = []
    for i in range(len(tps)):
        accuracy = (tps[i] + data_size - (fps[i] + fns[i])) / data_size
        accuracies.append(accuracy)
    macro_accuracy = sum(accuracies) / len(accuracies)
    return macro_accuracy


In [18]:
def evaluateKNN(classified_data, confusionMatrix):
    # Compute the confusion matrix
    confusionmatrix = confusion_matrix(classified_data)
    classified_data_size = len(classified_data)
    # Compute TP, FP, FN for each class
    tps = computeTPs(confusionmatrix)
    fps = computeFPs(confusionmatrix)
    fns = computeFNs(confusionmatrix)
    
    # Calculate precision, recall, f-measure, and accuracy using macro-average approach
    precision = computeMacroPrecision(tps, fps, fns, classified_data_size)
    recall = computeMacroRecall(tps, fps, fns, classified_data_size)
    f_measure = computeMacroFMeasure(recall,precision)
    accuracy = computeAccuracy(tps, fps, fns, classified_data_size)
    
    # Return the computed measures
    return precision, recall, f_measure, accuracy


In [19]:
precision, recall, f_measure, accuracy = evaluateKNN(classified_data , confusion_matrix)

In [20]:
accuracy

0.952988047808765

#  k-fold cross-validation technique

In [21]:
def splitDataForCrossValidation(training_data, f):
    folds = []
    fold_size = len(training_data) // f
    remainder = len(training_data) % f
    
    start_idx = 0
    for i in range(1, f + 1):
        fold_end = start_idx + fold_size
        if i <= remainder:
            fold_end += 1
        
        test_indices = []
        train_indices = []
        for idx in range(len(training_data)):
            if start_idx <= idx < fold_end:
                test_indices.append(idx)
            else:
                train_indices.append(idx)
        
        training_fold = training_data[train_indices]
        testing_fold = training_data[test_indices]
        
        folds.append([i, training_fold, testing_fold])
        start_idx = fold_end
    
    return folds


In [22]:
def validateDataFormat(data, f):
    # Check if the header is present
    header = ["Path", "ActualClass", "PredictedClass", "FoldNumber"]
    if not np.array_equal(data[0], header):
        return False
    
    # Check if the values in the "Path" column are file paths
    if any(not isinstance(path, str) for path in data[1:, 0]):
        return False
    
    # Check if the values in the "ActualClass" and "PredictedClass" columns are classes from the scheme
    # (Assuming the class scheme is defined somewhere)
    class_scheme = ['Female', 'Male', 'Primate', 'Rodent', 'Food'] # Replace with actual class scheme
    if any(cls not in class_scheme for cls in np.unique(data[1:, 1:3])):
        return False
    
    # Check if the values in the "FoldNumber" column are integers in [0,f) range
    if any(not isinstance(num, int) or num < 0 or num >= f for num in data[1:, 3]):
        return False
    
    # Check if there are as many entries in the "Path" column as there are in the other columns
    if not all(len(data[1:, 0]) == len(data[1:, i]) for i in range(1, 4)):
        return False
    
    # Check if the number of entries per each integer in [0,f) range for FoldNumber are approximately the same
    fold_counts = {i: 0 for i in range(f)}
    for fold_num in data[1:, 3]:
        fold_counts[fold_num] += 1
    max_count = max(fold_counts.values())
    min_count = min(fold_counts.values())
    if max_count - min_count > 1:
        return False
    
    return True


In [27]:
def evaluateCrossValidation(classified_data_list, evaluation_func = evaluateKNN):
    total_precision = 0
    total_recall = 0
    total_f_measure = 0
    total_accuracy = 0
    num_rounds = len(classified_data_list)

    # Iterate over each round's classified data
    for classified_data in classified_data_list:
        # Evaluate the classified data using the evaluation function
        precision, recall, f_measure, accuracy = evaluation_func(classified_data,confusion_matrix)

        # Accumulate the evaluation measures
        total_precision += precision
        total_recall += recall
        total_f_measure += f_measure
        total_accuracy += accuracy

    # Calculate average measures
    avg_precision = total_precision / num_rounds
    avg_recall = total_recall / num_rounds
    avg_f_measure = total_f_measure / num_rounds
    avg_accuracy = total_accuracy / num_rounds

    return avg_precision, avg_recall, avg_f_measure, avg_accuracy


In [28]:
def crossEvaluateKNN(training_data, k, measure_func, similarity_flag, f,knn_func= kNN ,split_func= splitDataForCrossValidation ):
    # Step 1: Split the data into folds
    folds = split_func(training_data, f)
    
    # Header for processed data
    processed = np.array([['Path', 'ActualClass', 'PredictedClass', 'FoldNumber']])
    # Step 2: Process each fold
    classified_list = []

    for fold in folds:
        fold_num , train_data, test_data = fold[0] ,fold[1] ,fold[2]

        classified_data = knn_func(train_data , 11 , computeMeasure2, False , test_data ,getMostCommonClass , getClassesOfKNearestNeighbours , readAndResize)        
        classified_list.append(classified_data)

        fold_numbers = np.full((len(classified_data), 1), fold_num)
        processed_fold = np.hstack((classified_data, fold_numbers))
        processed = np.append(processed, processed_fold, axis=0)

 
    avg_precision, avg_recall, avg_fMeasure, avg_accuracy = evaluateCrossValidation(classified_list ,evaluateKNN )

     # The measures are now added to the end. You should invoke validation BEFORE this step.
    h = ['avg_precision', 'avg_recall', 'avg_f_measure', 'avg_accuracy']
    v = [avg_precision, avg_recall, avg_fMeasure, avg_accuracy]

    processed = np.append(processed, [h], axis=0)
    processed = np.append(processed, [v], axis=0)

    return processed
    


In [29]:
processed = crossEvaluateKNN(train_ndarray , 11 , computeMeasure2 , False , 5 , kNN ,splitDataForCrossValidation  )

In [30]:
processed

array([['Path', 'ActualClass', 'PredictedClass', 'FoldNumber'],
       ['Path', 'ActualClass', 'PredictedClass', '1'],
       ['..\\Images/Student_Test/generated.photos_v3_0626369.jpg',
        'Female', 'Female', '1'],
       ...,
       ['..\\Images/Student_Test/30.jpg', 'Food', 'Rodent', '5'],
       ['avg_precision', 'avg_recall', 'avg_f_measure', 'avg_accuracy'],
       ['0.5781805555555556', '0.6802508129770668', '0.6250762700278367',
        '0.952988047808765']], dtype='<U70')